# Welcome to Sales Report
author: Jack Lau <br>
last edit: 05/11/19 <br>
purpose: this report is use to show the sales info for the intopia

In [1]:
import pandas as pd
from intopia_analysis import *

In [2]:
df_contact = pd.read_csv('Intopia - Contact List - Sheet1.csv')
df_sales = get_sales_data('phase2/period7/')
df_production = get_production_data('phase2/period7/')
df_inventory = get_inventory_data('phase2/period7/')
df_ad = get_advertising_data('phase2/period7/')

In [3]:
df_ad_fixed = df_ad[['Company', 'type', 'region', 'Total']]
price = (df_sales['Price'].replace( '[\$,)]','', regex=True )
                   .replace( '[(]','-',   regex=True ).astype(float))
df_sales['Price'] = price

In [4]:
df_supply = pd.merge(df_inventory, df_production, on=['Company', 'Grade', 'region', 'type'], how='outer')
df_supply = df_supply[['Company', 'type', 'Grade', 'region', 'Units', 'Unit Production']]
df_supply = df_supply.rename(columns={'Units':"inventory", 'Unit Production':"production"})
df_total = pd.merge(df_supply, df_sales, on=['Company', 'Grade', 'region', 'type'], how='outer')
df_total = df_total.rename(columns={'Unit Sales':"sales"})
df_total = pd.merge(df_total, df_ad_fixed, on=['Company', 'region', 'type'], how='outer')
df_total = df_total.rename(columns={'Total':"marketing"})

## Total Sales per area

In [12]:
df_sales['cost'] = df_sales['Price'] * df_sales['Unit Sales']

In [30]:
df_sales[(df_sales['region'] == 'cc') & (df_sales['type'] == 'x') & (df_sales['Grade'] == 0)]

,Company,Unit Sales,Grade,Price,type,region,cost
0,4,100.000,0,80.0,x,cc,8000.000
1,9,150.662,0,112.0,x,cc,16874.144
12,40,190.000,0,75.0,x,cc,14250.000
17,49,92.152,0,80.0,x,cc,7372.160
20,52,228.991,0,73.0,x,cc,16716.343
23,58,0.003,0,30.0,x,cc,0.090


In [22]:
df_sales_sum = pd.DataFrame(df_sales.groupby(['type', 'region','Grade'])['Unit Sales'].sum())
df_sales_cost = pd.DataFrame(df_sales.groupby(['type', 'region','Grade'])['cost'].sum())
df_sales_insight = pd.merge(df_sales_sum, df_sales_cost, on=['type', 'region', 'Grade'], how='outer')
df_sales_insight['average price'] = df_sales_insight['cost'] / df_sales_insight['Unit Sales']
df_sales_insight

Unit Sales        cost  average price
type region Grade                                       
x    cc     0         761.808   63212.737      82.977255
            1         569.606   52826.855      92.742799
            2        1289.493  132715.043     102.920328
            3         817.155   90897.132     111.236096
            4         120.000   10920.000      91.000000
     ec     0        2005.531  127514.949      63.581639
            1         582.726   48143.005      82.616882
            2        1064.344   89705.044      84.282003
            3         130.397   14897.620     114.248181
            4         120.000   10080.000      84.000000
     wc     0         657.904   44083.580      67.006098
            1        1079.043   96613.062      89.535878
            2        1744.485  170372.217      97.663332
            3         630.236   69982.083     111.041075
            4         260.000   24700.000      95.000000
y    cc     0         255.000   50750.000     199.019608
            2         590.000  185120.000     313.762712
            3         580.000  186300.000     321.206897
            4         271.000   83468.000     308.000000
     ec     0           9.000    1890.000     210.000000
            2          80.000   20000.000     250.000000
            4         168.939   48485.493     287.000000
     wc     0         245.991   49547.424     201.419662
            1         310.000   72900.000     235.161290
            2         739.000  221410.000     299.607578
            3         627.902  187715.286     298.956344
            4         123.000   36162.000     294.000000

## Total marketing as percentage of sales

In [44]:
df_sales_per_team = pd.DataFrame(df_sales.groupby(['Company', 'region','type'])['cost'].sum())
df_sales_per_team = pd.merge(df_sales_per_team, df_ad_fixed, on=['type', 'region', 'Company'], how='outer')
df_sales_per_team['percentage_sales'] = df_sales_per_team['Total'] / df_sales_per_team['cost']
df_sales_per_team = df_sales_per_team[['Company', 'region', 'type', 'percentage_sales']]

In [45]:
df_total = pd.merge(df_total, df_sales_per_team, on=['Company', 'region', 'type'], how='outer')

## Top sales in CC

In [51]:
df_total[(df_total['region'] == 'cc') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

,Company,type,Grade,region,inventory,production,sales,Price,marketing,percentage_sales
115,17,x,2.0,cc,56.587,NaN,343.413,100.00,1000.0,0.025444
100,48,x,3.0,cc,8.470,290.0,281.530,115.00,500.0,0.015444
101,48,x,3.0,cc,8.470,290.0,281.530,115.00,500.0,0.015444
89,9,x,3.0,cc,423.765,NaN,248.175,123.00,1092.0,0.023038
127,52,x,0.0,cc,181.009,NaN,228.991,73.00,15.0,0.000897
128,54,x,1.0,cc,12.520,NaN,212.480,97.00,NaN,NaN
121,34,x,2.0,cc,57.977,NaN,202.023,96.00,545.0,0.022809
126,40,x,0.0,cc,290.000,290.0,190.000,75.00,400.0,0.028070
125,40,x,0.0,cc,290.000,290.0,190.000,75.00,400.0,0.028070
114,14,x,3.0,cc,682.553,580.0,187.447,88.00,NaN,NaN


## Total Sales in WC 

In [50]:
df_total[(df_total['region'] == 'wc') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

,Company,type,Grade,region,inventory,production,sales,Price,marketing,percentage_sales
205,35,x,1.0,wc,NaN,NaN,360.000,82.00,1000.0,0.033875
71,22,x,2.0,wc,NaN,307.0,307.000,82.00,800.0,0.030983
59,56,x,3.0,wc,18.793,280.0,261.207,94.00,80.0,0.001624
58,56,x,4.0,wc,825.000,560.0,260.000,95.00,80.0,0.001624
46,12,x,2.0,wc,246.168,NaN,246.118,105.00,400.0,0.009242
66,17,x,2.0,wc,NaN,620.0,245.000,95.00,1250.0,0.039968
55,41,x,0.0,wc,75.000,75.0,235.000,61.52,120.0,0.008088
77,42,x,0.0,wc,NaN,NaN,233.877,75.00,100.0,0.003598
81,50,x,1.0,wc,484.212,300.0,225.503,94.00,100.0,0.004718
51,28,x,2.0,wc,290.000,NaN,200.000,115.00,720.0,0.016767


## Total Sales in EC 

In [49]:
df_total[(df_total['region'] == 'ec') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

,Company,type,Grade,region,inventory,production,sales,Price,marketing,percentage_sales
2,9,x,0.0,ec,351.932,NaN,878.561,71.0,3591.0,0.039525
201,44,x,0.0,ec,NaN,NaN,662.241,50.0,NaN,NaN
3,9,x,1.0,ec,NaN,NaN,335.000,85.0,3591.0,0.039525
198,20,x,2.0,ec,NaN,NaN,300.000,82.0,960.0,0.039024
16,22,x,2.0,ec,NaN,297.000,297.000,75.0,550.0,0.022916
11,17,x,2.0,ec,NaN,300.000,170.000,90.0,750.0,0.045045
157,56,y,4.0,ec,162.061,135.000,168.939,287.0,NaN,NaN
6,12,x,2.0,ec,NaN,NaN,149.586,100.0,400.0,0.018052
32,43,x,0.0,ec,85.754,225.000,139.246,72.0,20.0,0.001995
199,26,x,0.0,ec,NaN,NaN,135.084,55.0,NaN,NaN
